In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2



# import pandas as pd
# import io
# import boto3
# from boto3.dynamodb.conditions import Key
# import os
# from google_images_download import google_images_download
# import glob
# import PIL
# import json
# import numpy as np
# import pathlib
# import pickle

# import textacy
# import en_core_web_sm

import code

# Prepare data files to train the models with a new dataset

In [2]:
code

<function code.interact(banner=None, readfunc=None, local=None, exitmsg=None)>

## Inputs

In [ ]:
tblName = "ResourceDocuments"
nodeIdentifierName = "photosynthesis-photosynthesis-photosynthesis-biology"

termTxtToImage_flpth = 'data/photosynthesis'
text_flpth = os.path.join(termTxtToImage_flpth, 'text')
img_flpth =  os.path.join(termTxtToImage_flpth, 'images')
imageLog_fir='logs'

resourceDbName = 'dynamodb'
s3Bucket = "egm-bucket/TEXT_TO_IMAGE_DATA/data"

### Get Data from Resource Db: 
photosynthesis whole

In [ ]:
# Get Definitions for photosynthesis from dynamodb

## Connect to dynamodb
dynamodbClient = boto3.resource("dynamodb")
# client = boto3.client('dynamodb')
# display(client.describe_table(TableName=tblName))

## Connect to table with resources
resourceTbl = dynamodbClient.Table(tblName)
# display(resourceTbl.global_secondary_indexes)
display("NUmber of Items in ResourceDb: {}".format(resourceTbl.item_count))

### Load text into pandas 
- For data munging
    - stats
    - Duplicates

In [ ]:
response = resourceTbl.query(
    IndexName='NODE_IDENTIFIER-index',
    KeyConditionExpression=Key('NODE_IDENTIFIER').eq(nodeIdentifierName)
)

# Pass through pandas for some data munging
rsrc_df = pd.DataFrame(response["Items"])
print("Db Response Shape: {}".format(rsrc_df.shape))

rsrc_df.drop_duplicates(['RESOURCE'], keep='last', inplace=True)
rsrc_df.reset_index(drop=True)

print("Db Response Shape: {}".format(rsrc_df.shape))
print(rsrc_df.columns)

In [ ]:
rsrc_df["RESOURCE_SOURCE"]

### Create Labels for text to image model

In [ ]:
oxf = "The process by which green plants and some other organisms use sunlight to synthesize nutrients from carbon dioxide and water. Photosynthesis in plants generally involves the green pigment chlorophyll and generates oxygen as a by-product."


In [ ]:
rsrc_df.append(pd.DataFrame({"RESOURCE_SOURCE": [oxf]}, sort=False))

In [ ]:
def handle_missing_directories(directory_flpth):
    # Handle missing Directory
    if not os.path.exists(directory_flpth):
        
        os.makedirs(directory_flpth)
        print("Made new directory: {}".format(directory_flpth))
        # print(os.path.join(dirname, flpth))
    else:
        pass
    
    return

# Create text file for each doc - Each Doc maps to an image

## TODO: incoroporate number of labals per line
def labels_to_imageTxt_files(rsrc_df, trainingData_term, trainigData_flpth='../data'):
    
    # Handle if a data directory for a term exists e.g. data/photosynthesis
    dirname = os.path.abspath('')
    termData_flpth = os.path.join(dirname, trainigData_flpth)
    handle_missing_directories(termData_flpth)
    
    
    ### Move resource df to textacy
   
    # Load into textacy to delimit sentences
    img_labels = rsrc_df.to_dict(orient="records")
    text_stream, metadata_stream = textacy.io.split_records(img_labels, 'RESOURCE')

    # Load english model
    en = en_core_web_sm.load()
    labels_corpus = textacy.Corpus(lang=en, texts=text_stream, metadatas=metadata_stream)
    
    caption_filename_path = os.path.join(trainigData_flpth, "captions.pickle")
    
    # Loop through corpus and write document to flpth (s3)
    ''' Each doc in a corpus equals and image'''
    for ix, doc in enumerate(labels_corpus):
        print("Number of Sentences: {}".format(doc.n_sents))
        
        # Paths to directories (Where to write the text files)
        filename = "{}_{}.txt".format(trainingData_term, ix)
        path_to_file = "{}/{}".format(trainigData_flpth, filename)
        
        # Write captions for google images
        f =  open(path_to_file, 'w')
        
        # Parse Document into sentences
        for sent in doc.sents:
            caption = textacy.preprocess.preprocess_text(sent.text,
                                               lowercase=True,
                                               no_punct=True
                                              )
            # f.write(label+"\n" )
            f.write(caption+" " )
            
        f.close()
        
    return ix + 1 # Count using 1 as start

# process labels for images


trainingData_term = 'photosynthesis'
txt_trainingData_flpth='{}/text'.format(termTxtToImage_flpth, 'text')

numText_files = labels_to_imageTxt_files(rsrc_df, trainingData_term, txt_trainingData_flpth)
numText_files

### Download Images from google 

In [ ]:

def download_images(term ,img_args):
    
    # Download Images 
    response = google_images_download.googleimagesdownload()
    img_paths = response.download(img_args)
    
#     # Post Process google image results
#     for idx, f in enumerate(img_paths[term]):
        
#         # Open Google image resulst and conver to jpeg
#         img = PIL.Image.open(f)
#         img_filetype = img.format.lower()  # 'JPEG'
        
#         rgb_img = img.convert('RGB')
#         img.close()
        
#         # Make new filenme to allign with text file name
#         filename = "{}_{}.{}".format(trainingData_term, idx, 'jpg')
#         newfilepath_f = os.path.join(os.path.dirname(f), filename)
        
#         # Save and image
#         rgb_img.save(newfilepath_f)
#         os.remove(f)
    
    
    return response 

img_args = {"keywords":"sun",
             "format": "png",
              "limit": 100,
             "output_directory": 'data',
            "metadata": True,
            "image_directory": "photosynthesis/images",
            "no_download": False,
            "extract_metadata":True
            # "size":"icon"
           }

response = download_images(trainingData_term, img_args)

### 1. Inputs

In [ ]:
#import tigDataLoader.utiils as dataloader
import os
import json
import logging

In [ ]:
predictTerm = "photosynthesis"
dataTerm = "photosynthesis"

In [ ]:
# Base file io inputs
trainData_flpth = os.path.join('data', predictTerm)
text_flpth = os.path.join(trainData_flpth, 'text')
img_flpth =  os.path.join(trainData_flpth, 'images')

# Google imagae download metadata
imageLog_fir='logs'

In [ ]:
def transformText_to_captions(txtDoc, numCaptions_per_image=5, txtSplit_method='svo'):
    # Take textacy doc and converts to a list of captions for an image
    
    if txtSplit_method=='noun_chunks':
        split_list = list(textacy.extract.noun_chunks(txtDoc, drop_determiners=False, min_freq=1))
    if txtSplit_method=='svo':
        split_list = list(textacy.extract.subject_verb_object_triples(txtDoc))
    
    return split_list

### 2. Make captions for images

In [ ]:
# 1. Load text to textacy - each document is linked to an image
imgTxt_corpus = dataloader.df_to_corpus(rsrc_df)
imgTxt_corpus

In [ ]:
/AttnGAN/data/photosynthesis/captions.pickle

In [ ]:
imgTxt_corpus[1].text

In [ ]:
imgTxt_corpus.word_doc_freqs(normalize='lemma', weighting='count', as_strings=True)

In [ ]:
txtDoc = imgTxt_corpus[0]
txtDoc

In [ ]:
imgTxt_corpus


In [ ]:
imgTxt_corpus.get(lambda x: len(x)

In [ ]:
docSplit_list = transformText_to_captions(txtDoc, numCaptions_per_image=5, txtSplit_method='svo')

In [ ]:
list(textacy.extract.ngrams(txtDoc, n=5, filter_stops=False))

In [ ]:
txtDoc.text

In [ ]:
textacy.extract.acronyms_and_definitions(txtDoc)

### 3. Download images
- from google

In [ ]:
img_args = {"keywords": dataTerm,
             "format": "png",
              "limit": 100,
             "output_directory": 'data',
            "metadata": True,
            "image_directory": "{}/images/".format(predictTerm),
            "no_download": False,
            "extract_metadata":True,
            "type": "clipart"
           }
img_paths, response = dataloader.download_images(dataTerm, img_args)

In [ ]:
## load text into textacy
imgTxt_corpus = df_to_corpus()


In [ ]:
## Load image meta-data json 
with open('coco/logs/sun.json') as f:
    data = pickle.load(f)
    
data

In [ ]:
splitDir = 'train'
with open(os.path.join(termTxtToImage_flpth, splitDir, "filenames.pickle"), "rb") as pickF:
    tstFilenames = pickle.load(pickF)
    
display(type(tstFilenames),
        len(tstFilenames),
        tstFilenames)

In [ ]:
tstFilenames[-1:]

In [ ]:
splitDir = 'test'
with open(os.path.join(termTxtToImage_flpth, splitDir, "filenames.pickle"), "rb") as pickF:
    tstFilenames = pickle.load(pickF)
    
display(type(tstFilenames),
        len(tstFilenames),
        tstFilenames)

In [ ]:
with open("data/photosynthesis/train/filenames.pickle", "rb") as pickF:
    trFilenames = pickle.load(pickF)
    
display(type(trFilenames),
        len(trFilenames),
        trFilenames)

In [ ]:
commonalities = set(trFilenames) - (set(trFilenames) - set(tstFilenames))
commonalities

In [ ]:
os.path.dirname("/home/ec2-user/environment/AttnGAN/data/photosynthesis/images/5. photosynthesis.png")

In [ ]:
 
s3Client = boto3.client("s3")     
s3Client.Object('my-bucket-name', 'newfile.txt').put(Body=content)

In [ ]:
list(labels_corpus.docs[0].sents)

In [ ]:
import requests
r = requests.get("https://ssec.si.edu/stemvisions-blog/what-photosynthesis")

In [ ]:
r.content

In [ ]:
# Create Fake data
import shutil
import itertools

# Get a list of the original files f
txtDirFileNm_lst = glob.glob(text_flpth+"/**/*.txt", recursive=True)
imgDirFileNm_lst = glob.glob(img_flpth+"/**/*.jpg", recursive=True)

fileCombo_lst = list(itertools.product(txtDirFileNm_lst,imgDirFileNm_lst))

# for item in fileCombo_lst:
    
#     # Get text name
#     txtget_relfilename

# # # Create list paired tuple pairs
# # comboFlpth_lst = list(zip(txtDirFileNm_lst, imgDirFileNm_lst))
# # comboFlpth_lst

# # new_list = []
# # for k,v in comboFlpth_lst.iteritems():
# #     new_list.extend([x for x in combinations(v, 2)]) 
# # # for txtFile, imgFile in comboFlpth_dict:
# # #     # Make copies of files and rename
# # #     shutil.copyfile(txtFile, dst)
# # #     shutil.copyfile(imgFile, dst)

# # new_list

